In [4]:
%cd ..

C:\files\code\projects\dnn-studia\project


In [58]:
from pathlib import Path
import os
from config.structure import get_data_sources

dataset_info = get_data_sources(root_path=Path('.'))['stanford']
images_dir = dataset_info['data_dir'] / 'car_ims'
dataset_info

{'source': 'http://imagenet.stanford.edu/internal/car196/car_ims.tgz',
 'data_dir': WindowsPath('data/input/stanford'),
 'annotations': {'original_file_path': WindowsPath('data/input/stanford/devkit/cars_annos.mat'),
  'csv_file_path': WindowsPath('data/input/stanford/devkit/cars_annos.csv'),
  'source': 'https://ai.stanford.edu/~jkrause/cars/car_devkit.tgz'},
 'class_names': {'original_file_path': WindowsPath('data/input/stanford/devkit/cars_meta.mat'),
  'csv_file_path': WindowsPath('data/input/stanford/devkit/cars_meta.csv'),
  'source': 'https://ai.stanford.edu/~jkrause/cars/car_devkit.tgz'},
 'num_classes': 196,
 'in_channels': 3}

In [59]:
import pandas as pd
annotations = pd.read_csv(dataset_info['annotations']['csv_file_path'])
annotations

,relative_im_path,bbox_x1,bbox_y1,bbox_x2,bbox_y2,class,test
0,car_ims/000001.jpg,112,7,853,717,0,0
1,car_ims/000002.jpg,48,24,441,202,0,0
2,car_ims/000003.jpg,7,4,277,180,0,0
3,car_ims/000004.jpg,33,50,197,150,0,0
4,car_ims/000005.jpg,5,8,83,58,0,0
...,...,...,...,...,...,...,...
16180,car_ims/016181.jpg,38,36,375,234,195,1
16181,car_ims/016182.jpg,29,34,235,164,195,1
16182,car_ims/016183.jpg,25,32,587,359,195,1
16183,car_ims/016184.jpg,56,60,208,186,195,1


In [130]:
from PIL import Image
import numpy as np

train_files = annotations.loc[annotations.test == 0, 'relative_im_path']
result_array = np.empty(shape=(train_files.shape[0],8))
for idx, image_file_name in enumerate(train_files):
    image_fp = images_dir.parent / image_file_name
    
    image_rgb = Image.open(image_fp).convert('RGB')    
    rgb_mean = np.array(image_rgb).mean(axis=(0,1))
    rgb_sd = np.array(image_rgb).std(axis=(0,1))
    
    image_grayscale = Image.open(image_fp).convert('LA')   
    grayscale_mean = np.array(image_rgb).mean()
    grayscale_sd = np.array(image_rgb).std()
    
    result_array[idx] = [*rgb_mean, *rgb_sd, grayscale_mean, grayscale_sd]

In [114]:
result_array

array([[122.04870733, 121.6701088 , 106.03189468, ...,  63.53639535,
        116.58357027,  72.07696572],
       [154.36988287, 134.48514989, 107.02029978, ...,  91.50737889,
        131.95844418,  87.56570686],
       [167.01889796, 164.19386681, 155.35767687, ...,  66.98040657,
        162.19014721,  70.31320371],
       ...,
       [118.76636173, 115.07467877, 115.15308659, ...,  74.37616597,
        116.3313757 ,  75.73601518],
       [111.3096309 , 114.64388075, 101.29394053, ...,  87.70721664,
        109.08248406,  85.09188734],
       [141.79109524, 130.3557381 , 118.49995238, ...,  58.85885296,
        130.21559524,  61.96392373]])

In [131]:
result_array.shape

(8144, 8)

In [132]:
result_array.mean(axis=0) 

array([120.03696796, 117.33762399, 116.01268726,  67.98191606,
        67.77189476,  68.98854547, 117.79575974,  68.8953465 ])

In [133]:
result_array.mean(axis=0) / 255

array([0.47073321, 0.46014755, 0.45495171, 0.26659575, 0.26577214,
       0.27054332, 0.46194416, 0.27017783])